In [1]:
import argparse
import os
import csv
import json
import random
from tqdm import tqdm
import numpy as np
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from datasets import load_dataset, Dataset, DatasetDict, load_from_disk, concatenate_datasets
from transformers import logging
from generate_utility import *
from transformers.generation import GenerationConfig
from peft import PeftModel
# import bitsandbytes as bnb

logging.set_verbosity_error()

/scratch/ffaisal/dialect-copa/vnv/vnv_copa/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dict_file="../data/chakavian_dict.txt"

In [3]:
with open(dict_file, encoding="utf-8") as f:
            lines = f.read().splitlines()

In [15]:
cr_ck={}
for i,line in enumerate(lines):
    if line.endswith('\t'):
        cr_ck[line.lower()[:-1]]=lines[i+1].lower()

In [46]:
DATADIR='../data'
lang='copa-hr'
all_lines=[]
for split in ["train", "val","train-genx"]:
    with open(os.path.join(DATADIR,lang,f"{split}.jsonl"), encoding="utf-8") as f:
        lines = f.read().splitlines()
    line_dicts = [json.loads(line) for line in lines]
    for line in line_dicts:
        all_lines.append(line['premise'])
        all_lines.append(line['choice1'])
        all_lines.append(line['choice2'])

In [47]:
all_cr_words=list(cr_ck.keys())

In [48]:
ck_lines=[]
for i,line in enumerate(all_lines):
    line_flag=False
    words=line.split(' ')
    new_line_words=[]
    for word in words:
        if word.lower() in all_cr_words:
            new_line_words.append(cr_ck[word.lower()])
        else:
            new_line_words.append(word)
    ck_lines.append(' '.join(new_line_words))

In [49]:
hr_ck_line_dict={}
for line1,line2 in zip(all_lines, ck_lines):
    hr_ck_line_dict[line1]=line2
    

In [51]:
for split in ["train", "val","train-genx"]:
    new_line_dicts=[]
    lang='copa-hr'
    with open(os.path.join(DATADIR,lang,f"{split}.jsonl"), encoding="utf-8") as f:
        lines = f.read().splitlines()
    line_dicts = [json.loads(line) for line in lines]
    for l in line_dicts:
        new_line_dicts.append({
            'premise':hr_ck_line_dict[l['premise']],
            'choice1':hr_ck_line_dict[l['choice1']],
            'choice2':hr_ck_line_dict[l['choice2']],
            'question':l['question'],
            'idx':l['idx'],
            'label':l['label']
        })

    import json
    lang='copa-ck'
    dest_file=f"{DATADIR}/{lang}/{split}.jsonl"
    output_file = open(dest_file,'w',encoding='utf-8')
    for dic in new_line_dicts:
        json.dump(dic, output_file, ensure_ascii=False) 
        output_file.write("\n")
    output_file.close()